In [36]:
import os
import re
from typing import Dict, Tuple, Optional, Union
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb, Normalize
from matplotlib.colorbar import ColorbarBase
from pymatgen.core import Structure
from wulffpack import SingleCrystal


def extract_surface_energies_pbs(parent_dir: str) -> Dict[Tuple[int, int, int], float]:
    """Extract surface energies from a directory structure containing PBS output files.
    
    Args:
        parent_dir: Directory path containing subdirectories with PBS output files.
        
    Returns:
        Dictionary mapping Miller indices tuples to surface energy values in J/m^2.
    """
def extract_surface_energies_slurm(parent_dir: str) -> Dict[Tuple[int, int, int], float]:
    """Extract surface energies from a directory structure containing SLURM output files.
    
    Args:
        parent_dir: Path to the parent directory containing Miller index subdirectories
        
    Returns:
        Dictionary mapping Miller indices tuples to their corresponding surface energies in J/m^2
        
    Example:
        >>> energies = extract_surface_energies_slurm("/path/to/calculations")
        >>> print(energies[(1, 1, 1)])
        2.34
    """
    results = {}
    
    for dirname in (d for d in os.listdir(parent_dir) if d.startswith('[')):
        full_path = os.path.join(parent_dir, dirname)
        if not os.path.isdir(full_path):
            continue
            
        miller_match = re.search(r'\[(\d{3})\]', dirname)
        if not miller_match:
            continue
            
        miller_indices = miller_match.group(1)
        miller_tuple = tuple(int(miller_indices[i]) for i in range(3))
        
        for file in os.listdir(full_path):
            if file.startswith('slurm-') and file.endswith('.out'):
                slurm_path = os.path.join(full_path, file)
                try:
                    with open(slurm_path, 'r') as f:
                        content = f.read()
                        energy_match = re.search(r'Surface energy:\s*(\d+\.\d+)\s*J/m\^2', content)
                        if energy_match:
                            results[miller_tuple] = float(energy_match.group(1))
                            break
                except Exception as e:
                    print(f"Error reading {slurm_path}: {e}")
    
    return results


def extract_surface_energies(parent_dir: str, system: str = 'pbs') -> Dict[Tuple[int, int, int], float]:
    """Extract surface energies from a directory structure containing job output files.
    
    Args:
        parent_dir: Directory path containing subdirectories with output files.
        system: Job system type ('pbs' or 'slurm'). Defaults to 'pbs'.
        
    Returns:
        Dictionary mapping Miller indices tuples to surface energy values in J/m^2.
        
    Raises:
        ValueError: If system type is not recognized.
    """
    if system.lower() == 'pbs':
        return extract_surface_energies_pbs(parent_dir)
    elif system.lower() == 'slurm':
        return extract_surface_energies_slurm(parent_dir)
    else:
        raise ValueError(f"Unrecognized system type: {system}. Use 'pbs' or 'slurm'.")


def extract_surface_energies_pbs(parent_dir: str) -> Dict[Tuple[int, int, int], float]:
    results = {}
    
    for dirname in os.listdir(parent_dir):
        if not dirname.startswith('[') or not dirname.endswith(']'):
            continue
            
        full_path = os.path.join(parent_dir, dirname)
        if not os.path.isdir(full_path):
            continue
        
        try:
            miller_str = dirname.strip('[]')
            if len(miller_str) != 3:
                continue
            miller_tuple = tuple(int(miller_str[i]) for i in range(3))
            
            output_file = f"miller-{miller_str}.o"
            matching_files = [f for f in os.listdir(full_path) if f.startswith(output_file)]
            
            if matching_files:
                pbs_path = os.path.join(full_path, matching_files[0])
                try:
                    with open(pbs_path, 'r') as f:
                        content = f.read()
                        energy_match = re.search(r'Surface energy:\s*(\d+\.\d+)\s*J/m\^2', content)
                        if energy_match:
                            results[miller_tuple] = float(energy_match.group(1))
                except Exception as e:
                    print(f"Error reading {pbs_path}: {e}")
                        
        except Exception as e:
            print(f"Error processing directory {dirname}: {e}")
            continue
    
    return results


def simple_wulff(
    surface_energies: Dict[Tuple[int, int, int], float],
    bulk_path: str,
    output_path: Optional[str] = None,
    view_angles: Tuple[float, float, float] = (45, 45, 0),
    colorbar_x: float = 0.38,
    colorbar_y: float = 0.15,
    colorbar_width: float = 0.45
) -> SingleCrystal:
    """Create a Wulff shape visualization with surface energy-based coloring and colorbar.
    
    Args:
        surface_energies: Dictionary mapping Miller indices to surface energies.
        bulk_path: Path to bulk structure file.
        output_path: Optional path to save the visualization.
        view_angles: Tuple of angles (elevation, azimuth, roll) for viewing the shape.
        colorbar_x: X-position of the colorbar.
        colorbar_y: Y-position of the colorbar.
        colorbar_width: Width of the colorbar.
        
    Returns:
        SingleCrystal object representing the Wulff construction.
    """
    bulk_structure = Structure.from_file(bulk_path)
    bulk_structure_ase = bulk_structure.to_ase_atoms()
    particle = SingleCrystal(surface_energies, primitive_structure=bulk_structure_ase)
    
    facet_fractions = particle.facet_fractions
    active_surfaces = {miller: surface_energies[miller] 
                      for miller in facet_fractions.keys() if facet_fractions[miller] > 0}
    
    min_energy = min(active_surfaces.values())
    max_energy = max(active_surfaces.values())
    
    fig = plt.figure(figsize=(12, 10))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
    ax = fig.add_subplot(gs[0], projection='3d')
    
    color_order = sorted(active_surfaces.keys(), 
                        key=lambda x: (sum(x), x[0], x[1], x[2]))
    
    n_surfaces = len(color_order)
    colors = {}
    saturation_factor = 0.85
    
    for i, miller in enumerate(color_order):
        color_val = i / (n_surfaces - 1) if n_surfaces > 1 else 0.5
        rgb_color = plt.cm.viridis(color_val)[:3]
        hsv_color = rgb_to_hsv(rgb_color)
        hsv_color[1] *= saturation_factor
        rgb_color = hsv_to_rgb(hsv_color)
        colors[miller] = (*rgb_color, 1.0)
    
    particle.make_plot(ax, colors=colors, alpha=0.95)
    ax.view_init(*view_angles)
    ax.set_axis_off()
    
    legend_ax = fig.add_subplot(gs[1])
    legend_ax.axis('off')
    
    sorted_by_percentage = sorted(facet_fractions.items(),
                                key=lambda x: x[1],
                                reverse=True)
    
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor=colors[m], alpha=0.9)
                      for m, _ in sorted_by_percentage if m in colors]
    
    legend_labels = [f"({m[0]}{m[1]}{m[2]}) - {p*100:.1f}%" 
                    for m, p in sorted_by_percentage if m in colors]
    
    legend = legend_ax.legend(
        legend_elements,
        legend_labels,
        title="Miller Indices",
        loc='center left',
        bbox_to_anchor=(-0.2, 0.45),
        bbox_transform=legend_ax.transAxes,
        fontsize=15,
        title_fontsize=17,
        labelspacing=1.32,
        handletextpad=1.1,
        handlelength=1.65,
        frameon=True,
        edgecolor='lightgray'
    )
    
    cbar_ax = fig.add_axes([
        colorbar_x - (colorbar_width/2),
        colorbar_y,
        colorbar_width,
        0.04
    ])
    
    norm = Normalize(vmin=min_energy, vmax=max_energy)
    cbar = ColorbarBase(cbar_ax, cmap=plt.cm.viridis, norm=norm, orientation='horizontal')
    cbar.ax.tick_params(labelsize=12)
    
    fig.text(
        colorbar_x,
        colorbar_y - 0.06,
        "Surface Energy (J/m²)",
        fontsize=14,
        ha='center'
    )
    
    plt.subplots_adjust(left=0.1, right=0.9, top=0.95, bottom=0.15)
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()
    
    return particle


def main() -> None:
    """Main execution function for processing surface energies and generating Wulff construction visualisation."""
    SURFACES_DIR = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production/Neutral"
    BULK_PATH = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/analysis/Li.cif"
    OUTPUT_PATH = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/analysis/Wulff_shapes/neutral_wulff.png"
    SYSTEM_TYPE = "slurm" # or "slurm"
    
    surface_energies = extract_surface_energies(SURFACES_DIR, system=SYSTEM_TYPE)
    print(surface_energies)
    if surface_energies:
        particle = simple_wulff(
            surface_energies=surface_energies,
            bulk_path=BULK_PATH,
            output_path=OUTPUT_PATH
        )
    else:
        print("No surface energies found in the directories")


if __name__ == "__main__":
    main()

{(1, 1, 1): 0.5335, (2, 1, 0): 0.4976, (3, 1, 0): 0.4932, (1, 0, 0): 0.469358, (3, 3, 2): 0.5143, (3, 3, 1): 0.5225, (3, 2, 0): 0.5011, (3, 2, 2): 0.5291, (3, 2, 1): 0.5274, (2, 2, 1): 0.5176, (3, 1, 1): 0.5172, (2, 1, 1): 0.5311, (1, 1, 0): 0.5078}


/opt/homebrew/Caskroom/miniconda/base/envs/workflow/lib/python3.12/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['rotations']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


In [40]:
import os
import re
from typing import Dict, Tuple, Optional, Union
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb, Normalize
from matplotlib.colorbar import ColorbarBase
from pymatgen.core import Structure
from wulffpack import SingleCrystal



def simple_wulff(
    surface_energies: Dict[Tuple[int, int, int], float],
    bulk_path: str,
    output_path: Optional[str] = None,
    view_angles: Tuple[float, float, float] = (45, 45, 0),
    colorbar_x: float = 0.38,
    colorbar_y: float = 0.15,
    colorbar_width: float = 0.45
) -> SingleCrystal:
    """Create a Wulff shape visualization with surface energy-based coloring and colorbar.
    
    Args:
        surface_energies: Dictionary mapping Miller indices to surface energies.
        bulk_path: Path to bulk structure file.
        output_path: Optional path to save the visualization.
        view_angles: Tuple of angles (elevation, azimuth, roll) for viewing the shape.
        colorbar_x: X-position of the colorbar.
        colorbar_y: Y-position of the colorbar.
        colorbar_width: Width of the colorbar.
        
    Returns:
        SingleCrystal object representing the Wulff construction.
    """
    bulk_structure = Structure.from_file(bulk_path)
    bulk_structure_ase = bulk_structure.to_ase_atoms()
    particle = SingleCrystal(surface_energies, primitive_structure=bulk_structure_ase, tol=1e-10, symprec=1e-10)
    
    facet_fractions = particle.facet_fractions
    active_surfaces = {miller: surface_energies[miller] 
                      for miller in facet_fractions.keys() if facet_fractions[miller] > 0}
    
    min_energy = min(active_surfaces.values())
    max_energy = max(active_surfaces.values())
    
    fig = plt.figure(figsize=(12, 10))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
    ax = fig.add_subplot(gs[0], projection='3d')
    
    color_order = sorted(active_surfaces.keys(), 
                        key=lambda x: (sum(x), x[0], x[1], x[2]))
    
    n_surfaces = len(color_order)
    colors = {}
    saturation_factor = 0.85
    
    for i, miller in enumerate(color_order):
        color_val = i / (n_surfaces - 1) if n_surfaces > 1 else 0.5
        rgb_color = plt.cm.viridis(color_val)[:3]
        hsv_color = rgb_to_hsv(rgb_color)
        hsv_color[1] *= saturation_factor
        rgb_color = hsv_to_rgb(hsv_color)
        colors[miller] = (*rgb_color, 1.0)
    
    particle.make_plot(ax, colors=colors, alpha=0.95)
    ax.view_init(*view_angles)
    ax.set_axis_off()
    
    legend_ax = fig.add_subplot(gs[1])
    legend_ax.axis('off')
    
    sorted_by_percentage = sorted(facet_fractions.items(),
                                key=lambda x: x[1],
                                reverse=True)
    
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor=colors[m], alpha=0.9)
                      for m, _ in sorted_by_percentage if m in colors]
    
    legend_labels = [f"({m[0]}{m[1]}{m[2]}) - {p*100:.1f}%" 
                    for m, p in sorted_by_percentage if m in colors]
    
    legend = legend_ax.legend(
        legend_elements,
        legend_labels,
        title="Miller Indices",
        loc='center left',
        bbox_to_anchor=(-0.2, 0.45),
        bbox_transform=legend_ax.transAxes,
        fontsize=15,
        title_fontsize=17,
        labelspacing=1.32,
        handletextpad=1.1,
        handlelength=1.65,
        frameon=True,
        edgecolor='lightgray'
    )
    
    cbar_ax = fig.add_axes([
        colorbar_x - (colorbar_width/2),
        colorbar_y,
        colorbar_width,
        0.04
    ])
    
    norm = Normalize(vmin=min_energy, vmax=max_energy)
    cbar = ColorbarBase(cbar_ax, cmap=plt.cm.viridis, norm=norm, orientation='horizontal')
    cbar.ax.tick_params(labelsize=12)
    
    fig.text(
        colorbar_x,
        colorbar_y - 0.06,
        "Surface Energy (J/m²)",
        fontsize=14,
        ha='center'
    )
    
    plt.subplots_adjust(left=0.1, right=0.9, top=0.95, bottom=0.15)
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()
    
    return particle


def main() -> None:
    """Main execution function for processing surface energies and generating Wulff construction visualisation."""
    BULK_PATH = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/analysis/Li.cif"
    OUTPUT_PATH = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/analysis/Wulff_shapes/vacuum_wulff.png"
    
    surface_energies = {(1, 0, 0): 0.4616, (3, 1, 0): 0.4935, (1, 1, 0): 0.4945, (3, 2, 0): 0.4974, (2, 1, 0): 0.5017, (3, 3, 1): 0.5176, (3, 3, 2): 0.5200, (2, 2, 1): 0.5210, (3, 1, 1): 0.5223, (3, 2, 1): 0.5286, (3, 2, 2): 0.5351, (2, 1, 1): 0.5376, (1, 1, 1): 0.5404, }
    print(surface_energies)
    if surface_energies:
        particle = simple_wulff(
            surface_energies=surface_energies,
            bulk_path=BULK_PATH,
            output_path=OUTPUT_PATH
        )
    else:
        print("No surface energies found in the directories")


if __name__ == "__main__":
    main()

{(1, 0, 0): 0.4616, (3, 1, 0): 0.4935, (1, 1, 0): 0.4945, (3, 2, 0): 0.4974, (2, 1, 0): 0.5017, (3, 3, 1): 0.5176, (3, 3, 2): 0.52, (2, 2, 1): 0.521, (3, 1, 1): 0.5223, (3, 2, 1): 0.5286, (3, 2, 2): 0.5351, (2, 1, 1): 0.5376, (1, 1, 1): 0.5404}


/opt/homebrew/Caskroom/miniconda/base/envs/workflow/lib/python3.12/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['rotations']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


In [20]:
import os
import re
import pandas as pd
from ase.io import read
from typing import Dict, List, Optional, Union

def get_pbs_energy(path: str, miller_str: str) -> Optional[float]:
    """Extract surface energy from PBS output files.
    
    Args:
        path: Directory path containing PBS output files
        miller_str: Miller index string without brackets
        
    Returns:
        Surface energy value in J/m^2 if found, None otherwise
    """
    output_file = f"miller-{miller_str}.o"
    matching_files = [f for f in os.listdir(path) if f.startswith(output_file)]
    
    if matching_files:
        pbs_path = os.path.join(path, matching_files[0])
        try:
            with open(pbs_path, 'r') as f:
                content = f.read()
                energy_match = re.search(r'Surface energy:\s*([-]?\d+\.\d+)\s*J/m\^2', content)
                if energy_match:
                    return float(energy_match.group(1))
        except Exception as e:
            print(f"Error reading {pbs_path}: {e}")
    return None

def get_slurm_energy(path: str) -> Optional[float]:
    """Extract surface energy from SLURM output files.
    
    Args:
        path: Directory path containing SLURM output files
        
    Returns:
        Surface energy value in J/m^2 if found, None otherwise
    """
    for file in os.listdir(path):
        if file.startswith('slurm-') and file.endswith('.out'):
            slurm_path = os.path.join(path, file)
            try:
                with open(slurm_path, 'r') as f:
                    content = f.read()
                    energy_match = re.search(r'Surface energy:\s*([-]?\d+\.\d+)\s*J/m\^2', content)
                    if energy_match:
                        return float(energy_match.group(1))
            except Exception as e:
                print(f"Error reading {slurm_path}: {e}")
    return None

def create_surface_energy_table(base_dir: str) -> pd.DataFrame:
    """Generate a DataFrame containing surface energies and PZC values for different facets.
    
    Args:
        base_dir: Base directory containing voltage subdirectories
        
    Returns:
        DataFrame with facet indices, PZC values, and surface energies at different voltages
    """
    data = []
    voltage_values = [-2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0]
    neutral_dir = os.path.join(base_dir, 'Neutral')
    miller_indices = sorted([d for d in os.listdir(neutral_dir) if d.startswith('[')])
    
    for miller in miller_indices:
        row = {'Facet': miller}
        miller_str = miller.strip('[]')
        
        try:
            pwo_path = os.path.join(neutral_dir, miller, 'Surface', 'espresso.pwo')
            atoms = read(pwo_path)
            row['PZC'] = atoms.calc.get_fermi_level()
        except Exception as e:
            row['PZC'] = None
            
        for v in voltage_values:
            col = f"{v}_VLi"
            row[col] = None
            dir_name = 'Neutral' if v == 0.0 else f"[{v}V]"
            voltage_dir = os.path.join(base_dir, dir_name)
            
            if os.path.exists(voltage_dir):
                miller_path = os.path.join(voltage_dir, miller)
                if os.path.exists(miller_path):
                    energy = get_pbs_energy(miller_path, miller_str) if v in [0.5, 1.0] else get_slurm_energy(miller_path)
                    row[col] = energy
                    
        data.append(row)
    
    df = pd.DataFrame(data)
    cols = ['Facet', 'PZC'] + [f'{v}_VLi' for v in voltage_values]
    return df[cols]

if __name__ == "__main__":
    base_dir = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production"
    df = create_surface_energy_table(base_dir)
    print("\nLi Surface Energies and PZC Values:")
    print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
    df.to_csv("li_surface_energies.csv", index=False)


Li Surface Energies and PZC Values:
Facet     PZC  -2.0_VLi  -1.5_VLi  -1.0_VLi  -0.5_VLi  0.0_VLi  0.5_VLi  1.0_VLi  1.5_VLi  2.0_VLi
[100] -2.6593   -0.1435    0.2209    0.4429    0.4561   0.4694   0.2292   0.0505  -0.1611  -0.4008
[110] -2.8050   -0.1641    0.2057    0.4507    0.5052   0.5078   0.2950   0.1338  -0.0553  -0.2693
[111] -2.7678   -0.0947    0.2642    0.4805    0.5267   0.5335   0.2620   0.0529  -0.2089  -0.5194
[210] -2.6982   -0.1373    0.2327    0.4674    0.4887   0.4976   0.2498   0.0576  -0.1729  -0.4361
[211] -2.7118   -0.1283    0.2242    0.4688    0.5212   0.5311   0.2885   0.0932  -0.1378  -0.3996
[221] -2.6940   -0.1153    0.2545    0.4898    0.5056   0.5176   0.2561   0.0593  -0.1731  -0.4330
[310] -2.6787   -0.1308    0.2312    0.4601    0.4822   0.4932   0.2450   0.0464  -0.1943  -0.4733
[311] -2.7203   -0.1038    0.2576    0.4857    0.5141   0.5172   0.2455   0.0344  -0.2228  -0.5137
[320] -2.7060   -0.1460    0.2316    0.4645    0.4897   0.5011   0.2441 

In [16]:
import os
import re
import pandas as pd
from typing import Dict, Tuple

def extract_surface_energies_slurm(parent_dir: str) -> Dict[Tuple[int, int, int], float]:
   results = {}
   for dirname in (d for d in os.listdir(parent_dir) if d.startswith('[')):
       full_path = os.path.join(parent_dir, dirname)
       if not os.path.isdir(full_path):
           continue
       miller_match = re.search(r'\[(\d{3})\]', dirname)
       if not miller_match:
           continue
       miller_indices = miller_match.group(1)
       miller_tuple = tuple(int(miller_indices[i]) for i in range(3))
       
       for file in os.listdir(full_path):
           if file.startswith('slurm-') and file.endswith('.out'):
               slurm_path = os.path.join(full_path, file)
               try:
                   with open(slurm_path, 'r') as f:
                       content = f.read()
                       energy_match = re.search(r'Surface energy:\s*(\d+\.\d+)\s*J/m\^2', content)
                       if energy_match:
                           results[miller_tuple] = round(float(energy_match.group(1)), 4)
                           break
               except Exception as e:
                   print(f"Error reading {slurm_path}: {e}")
   return results

def create_miller_tuple(miller_str: str) -> Tuple[int, int, int]:
   miller_int = int(float(miller_str))
   return (miller_int//100, (miller_int//10)%10, miller_int%10)

def create_comparison_table(neutral_dir: str, vacuum_file: str) -> pd.DataFrame:
   solvated_energies = extract_surface_energies_slurm(neutral_dir)
   df_vacuum = pd.read_csv(vacuum_file, sep='\t', skiprows=0)
   
   data = []
   delta_gammas = []
   for _, row in df_vacuum.iterrows():
       miller_str = str(row['Slab'])
       miller_tuple = create_miller_tuple(miller_str)
       
       if miller_tuple in solvated_energies:
           solv_energy = solvated_energies[miller_tuple]
           vac_energy = row['Surface Energy (J/m^2)']
           delta_gamma = round(solv_energy - vac_energy, 4)
           delta_gammas.append(abs(delta_gamma))
           
           data.append({
               'Miller Index': f"[{miller_str.split('.')[0]}]",
               'Vacuum γ (J/m²)': vac_energy,
               'Solvated γ (J/m²)': solv_energy,
               'Δγ (J/m²)': delta_gamma
           })
   
   df = pd.DataFrame(data).sort_values('Miller Index')
   abs_range = round(max(delta_gammas) - min(delta_gammas), 4)
   print(f"\nAbsolute Δγ range: {abs_range} J/m²")
   print(f"Maximum absolute Δγ: {max(delta_gammas)} J/m²")
   print(f"Minimum absolute Δγ: {min(delta_gammas)} J/m²")
   return df

neutral_dir = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production/Neutral"
vacuum_file = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production/Vacuum/surface_energies.txt"

comparison_table = create_comparison_table(neutral_dir, vacuum_file)
print("\nSurface Energy Comparison Table:")
print(comparison_table.to_string(index=False))
comparison_table.to_csv("surface_energy_comparison.csv", index=False)


Absolute Δγ range: 0.013 J/m²
Maximum absolute Δγ: 0.0133 J/m²
Minimum absolute Δγ: 0.0003 J/m²

Surface Energy Comparison Table:
Miller Index  Vacuum γ (J/m²)  Solvated γ (J/m²)  Δγ (J/m²)
       [100]           0.4616             0.4694     0.0078
       [110]           0.4945             0.5078     0.0133
       [111]           0.5404             0.5335    -0.0069
       [210]           0.5017             0.4976    -0.0041
       [211]           0.5376             0.5311    -0.0065
       [221]           0.5210             0.5176    -0.0034
       [310]           0.4935             0.4932    -0.0003
       [311]           0.5223             0.5172    -0.0051
       [320]           0.4974             0.5011     0.0037
       [321]           0.5286             0.5274    -0.0012
       [322]           0.5351             0.5291    -0.0060
       [331]           0.5176             0.5225     0.0049
       [332]           0.5200             0.5143    -0.0057


In [26]:
import os
from ase.io import read
import pandas as pd
from typing import Dict, Tuple

def get_fermi_levels(directory: str) -> Dict[str, float]:
    """Extract Fermi levels from neutral and vacuum calculations."""
    results = {}
    
    # Process Neutral directory
    neutral_dir = os.path.join(directory, 'Neutral')
    for dirname in (d for d in os.listdir(neutral_dir) if d.startswith('[')):
        try:
            pwo_path = os.path.join(neutral_dir, dirname, 'Surface', 'espresso.pwo')
            atoms = read(pwo_path)
            results[f"neutral_{dirname}"] = atoms.calc.get_fermi_level()
        except Exception as e:
            print(f"Error reading neutral {dirname}: {e}")
    
    # Process Vacuum directory
    vacuum_dir = os.path.join(directory, 'Vacuum')
    for dirname in os.listdir(vacuum_dir):
        if dirname.startswith('Surface_'):
            try:
                miller = dirname.split('_')[1]
                pwo_path = os.path.join(vacuum_dir, dirname, 'espresso.pwo')
                atoms = read(pwo_path)
                results[f"vacuum_{miller}"] = atoms.calc.get_fermi_level()
            except Exception as e:
                print(f"Error reading vacuum {dirname}: {e}")
    
    return results

def create_fermi_comparison(base_dir: str) -> pd.DataFrame:
    fermi_levels = get_fermi_levels(base_dir)
    data = []
    delta_fermis = []
    
    for miller in sorted(set(key.split('_')[1] for key in fermi_levels.keys() if key.startswith('neutral_'))):
        neutral_key = f"neutral_{miller}"
        vacuum_key = f"vacuum_{miller.strip('[]')}"
        
        if neutral_key in fermi_levels and vacuum_key in fermi_levels:
            neutral_fermi = fermi_levels[neutral_key]
            vacuum_fermi = fermi_levels[vacuum_key]
            delta = round(neutral_fermi - vacuum_fermi, 4)
            delta_fermis.append(abs(delta))
            
            data.append({
                'Miller Index': miller,
                'Vacuum E_f (eV)': vacuum_fermi,
                'Solvated E_f (eV)': neutral_fermi,
                'ΔE_f (eV)': delta
            })
    
    df = pd.DataFrame(data).sort_values('Miller Index')
    
    abs_range = round(max(delta_fermis) - min(delta_fermis), 4)
    print(f"\nAbsolute ΔE_f range: {abs_range} eV")
    print(f"Maximum absolute ΔE_f: {max(delta_fermis)} eV")
    print(f"Minimum absolute ΔE_f: {min(delta_fermis)} eV")
    
    return df

if __name__ == "__main__":
    base_dir = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production"
    comparison_table = create_fermi_comparison(base_dir)
    print("\nFermi Level Comparison Table:")
    print(comparison_table.to_string(index=False))
    comparison_table.to_csv("fermi_level_comparison.csv", index=False)


Absolute ΔE_f range: 1.4043 eV
Maximum absolute ΔE_f: 1.8155 eV
Minimum absolute ΔE_f: 0.4112 eV

Fermi Level Comparison Table:
Miller Index  Vacuum E_f (eV)  Solvated E_f (eV)  ΔE_f (eV)
       [100]          -2.0942            -2.6593    -0.5651
       [110]          -2.3938            -2.8050    -0.4112
       [111]          -1.7637            -2.7678    -1.0041
       [210]          -1.8855            -2.6982    -0.8127
       [211]          -1.9212            -2.7118    -0.7906
       [221]          -1.5168            -2.6940    -1.1772
       [310]          -1.5709            -2.6787    -1.1078
       [311]          -1.5686            -2.7203    -1.1517
       [320]          -1.6281            -2.7060    -1.0779
       [321]          -1.6689            -2.7854    -1.1165
       [322]          -0.8744            -2.6822    -1.8078
       [331]          -0.9491            -2.7252    -1.7761
       [332]          -0.8943            -2.7098    -1.8155


In [25]:
import os
from ase.io import read
import numpy as np

def calculate_pzc_differences(base_dir: str, reference_value: float = -3.404):
    """Calculate PZC differences (reference - fermi) for each facet."""
    neutral_dir = os.path.join(base_dir, 'Neutral')
    miller_indices = sorted([d for d in os.listdir(neutral_dir) if d.startswith('[')])
    
    pzc_diffs = {}
    for miller in miller_indices:
        try:
            pwo_path = os.path.join(neutral_dir, miller, 'Surface', 'espresso.pwo')
            atoms = read(pwo_path)
            fermi_level = atoms.calc.get_fermi_level()
            pzc_diffs[miller] = reference_value - fermi_level
        except Exception as e:
            print(f"Error processing {miller}: {e}")
    
    # Find smallest and largest differences
    min_diff_idx = min(pzc_diffs.items(), key=lambda x: abs(x[1]))
    max_diff_idx = max(pzc_diffs.items(), key=lambda x: abs(x[1]))
    
    print("\nPZC Differences (Reference - Fermi):")
    for miller, diff in pzc_diffs.items():
        print(f"{miller}: {diff:.4f} V_{{Li/Li+}}")
    
    print(f"\nThe PZC differences from Li/Li+ reference range from {min(pzc_diffs.values()):.4f} to {max(pzc_diffs.values()):.4f} V_{{Li/Li+}}")
    print(f"Smallest difference: {min_diff_idx[0]} at {min_diff_idx[1]:.4f} V_{{Li/Li+}}")
    print(f"Largest difference: {max_diff_idx[0]} at {max_diff_idx[1]:.4f} V_{{Li/Li+}}")
    
    return pzc_diffs

if __name__ == "__main__":
    base_dir = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production"
    pzc_diffs = calculate_pzc_differences(base_dir)


PZC Differences (Reference - Fermi):
[100]: -0.7447 V_{Li/Li+}
[110]: -0.5990 V_{Li/Li+}
[111]: -0.6362 V_{Li/Li+}
[210]: -0.7058 V_{Li/Li+}
[211]: -0.6922 V_{Li/Li+}
[221]: -0.7100 V_{Li/Li+}
[310]: -0.7253 V_{Li/Li+}
[311]: -0.6837 V_{Li/Li+}
[320]: -0.6980 V_{Li/Li+}
[321]: -0.6186 V_{Li/Li+}
[322]: -0.7218 V_{Li/Li+}
[331]: -0.6788 V_{Li/Li+}
[332]: -0.6942 V_{Li/Li+}

The PZC differences from Li/Li+ reference range from -0.7447 to -0.5990 V_{Li/Li+}
Smallest difference: [110] at -0.5990 V_{Li/Li+}
Largest difference: [100] at -0.7447 V_{Li/Li+}


In [17]:
E_vac = 0.93 
E_fermi = -2.8050
shift = 4.44
work_function = E_fermi - E_vac
NHE = work_function -  shift
print("NHE", NHE)

Li_ref = -3.404 
Li_vs_NHE = NHE - Li_ref

print("Li_vs_NHE", Li_vs_NHE)
experimental = -3.04
delta = experimental - Li_vs_NHE
print("delta", delta)


NHE -8.175
Li_vs_NHE -4.771000000000001
delta 1.7310000000000008
